In [1]:
import pandas as pd 
enriquecido_df = pd.read_csv('gnn_recommender.csv')


In [70]:
# Opcional 
enriquecido_df = enriquecido_df.sample(frac=0.7, random_state=42)


In [72]:


# 1. Extraer la información de nodos (parlamentarios)
# Nos quedamos con las columnas relevantes para los parlamentarios (person_1 y person_2 con sus descripciones)
nodos_1 = enriquecido_df[['parliamentarian_1', 'biografia_1', 'region_1', 'partido_1', 'sector_1', 'fuente_1' , 'url_imagen_1']]
nodos_2 = enriquecido_df[['parliamentarian_2', 'biografia_2', 'region_2', 'partido_2', 'sector_2', 'fuente_2', 'url_imagen_2']]

# Renombrar las columnas de nodos_2 para que coincidan con nodos_1
nodos_2.columns = ['parliamentarian_1', 'biografia_1', 'region_1', 'partido_1', 'sector_1', 'fuente_1', 'url_imagen_1']

# Concatenar los nodos de ambos parlamentarios (quitar duplicados)
nodos = pd.concat([nodos_1, nodos_2], axis=0).drop_duplicates(subset='parliamentarian_1').reset_index(drop=True)


# 2. Extraer la información de aristas (interacciones)
# Nos quedamos con las columnas relevantes para las interacciones entre parlamentarios
aristas = enriquecido_df[['parliamentarian_1', 'parliamentarian_2', 'proportion_agreement', 'region_1', 'region_2', 'partido_1', 'partido_2', 'sector_1', 'sector_2']]
aristas = aristas.drop_duplicates(subset=['parliamentarian_1', 'parliamentarian_2']).reset_index(drop=True)



In [73]:
import os.path as osp
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from torch_geometric.data import Data
import torch_geometric.transforms as T
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F
from torch.optim import Adam
from sklearn.model_selection import train_test_split

# Paso 1: One-Hot Encoding de las columnas categóricas 'region_1', 'partido_1', 'sector_1'
one_hot_features = pd.get_dummies(nodos[['region_1', 'partido_1', 'sector_1']])
one_hot_tensor = torch.tensor(one_hot_features.values, dtype=torch.float)

# Paso 2: Generar embeddings para las biografías usando SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
with torch.no_grad():
    bio_embeddings = model.encode(nodos['biografia_1'].tolist(), convert_to_tensor=True, show_progress_bar=True)
    bio_embeddings = bio_embeddings.cpu()

# Paso 3: Concatenar las características (One-Hot + Embeddings)
node_features = torch.cat([one_hot_tensor, bio_embeddings], dim=-1)

# Crear mapeo de IDs para los parlamentarios
unique_parliamentarians = pd.concat([aristas['parliamentarian_1'], aristas['parliamentarian_2']]).unique()
parliamentarian_to_index = {name: idx for idx, name in enumerate(unique_parliamentarians)}

# Mapear los IDs en las aristas
aristas['source'] = aristas['parliamentarian_1'].map(parliamentarian_to_index)
aristas['target'] = aristas['parliamentarian_2'].map(parliamentarian_to_index)

# Crear el edge_index para el grafo
edge_index = torch.tensor([aristas['source'].values, aristas['target'].values], dtype=torch.long)

# Usar 'proportion_agreement' como etiquetas
edge_label = torch.tensor(aristas['proportion_agreement'].values, dtype=torch.float)

# Paso 4: Dividir los datos manualmente usando train_test_split
train_edges, temp_edges = train_test_split(aristas, test_size=0.30, random_state=42)
val_edges, test_edges = train_test_split(temp_edges, test_size=0.50, random_state=42)

# Extraer los edge_index y las etiquetas (valores continuos)
train_edge_index = torch.tensor([train_edges['source'].values, train_edges['target'].values], dtype=torch.long)
val_edge_index = torch.tensor([val_edges['source'].values, val_edges['target'].values], dtype=torch.long)
test_edge_index = torch.tensor([test_edges['source'].values, test_edges['target'].values], dtype=torch.long)

train_edge_label = torch.tensor(train_edges['proportion_agreement'].values, dtype=torch.float)
val_edge_label = torch.tensor(val_edges['proportion_agreement'].values, dtype=torch.float)
test_edge_label = torch.tensor(test_edges['proportion_agreement'].values, dtype=torch.float)

# Crear los objetos Data para cada conjunto
train_data = Data(x=node_features, edge_index=train_edge_index, edge_label=train_edge_label)
val_data = Data(x=node_features, edge_index=val_edge_index, edge_label=val_edge_label)
test_data = Data(x=node_features, edge_index=test_edge_index, edge_label=test_edge_label)

# Paso 5: Definir el modelo GNN
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv(-1, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = torch.nn.Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = torch.nn.Linear(hidden_channels, 1)

    def forward(self, z, edge_index):
        row, col = edge_index
        z = torch.cat([z[row], z[col]], dim=-1)
        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)

class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x, edge_index):
        z = self.encoder(x, edge_index)
        return self.decoder(z, edge_index)

# Paso 6: Entrenamiento del modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(hidden_channels=32).to(device)
optimizer = Adam(model.parameters(), lr=0.01)

def train():
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x, train_data.edge_index)  # Se usa edge_index en lugar de edge_label_index
    target = train_data.edge_label
    loss = F.mse_loss(pred, target)
    loss.backward()
    optimizer.step()
    return float(loss)

@torch.no_grad()
def test(data):
    data = data.to(device)
    model.eval()
    pred = model(data.x, data.edge_index)  # Se usa edge_index en lugar de edge_label_index
    target = data.edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse)
@torch.no_grad()
def predict(data):
    data = data.to(device)
    model.eval()
    pred = model(data.x, data.edge_index)  # Predicciones del modelo
    target = data.edge_label.float()  # Valores reales
    return pred.cpu().numpy(), target.cpu().numpy()  # Devuelve como numpy arrays para facilitar su uso en Pandas

# Entrenar el modelo por 300 épocas
for epoch in range(1, 301):
    train_data = train_data.to(device)
    loss = train()
    train_rmse = test(train_data)
    val_rmse = test(val_data)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, Val: {val_rmse:.4f}')


c:\Users\benja\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 001, Loss: 2.8556, Train: 1.5373, Val: 1.5363
Epoch: 002, Loss: 2.3632, Train: 1.1228, Val: 1.1403
Epoch: 003, Loss: 1.2608, Train: 0.3849, Val: 0.4316
Epoch: 004, Loss: 0.1481, Train: 1.3926, Val: 1.2960
Epoch: 005, Loss: 1.9392, Train: 0.6852, Val: 0.6339
Epoch: 006, Loss: 0.4695, Train: 0.3680, Val: 0.4119
Epoch: 007, Loss: 0.1354, Train: 0.6910, Val: 0.7271
Epoch: 008, Loss: 0.4775, Train: 0.8616, Val: 0.8900
Epoch: 009, Loss: 0.7424, Train: 0.8992, Val: 0.9260
Epoch: 010, Loss: 0.8085, Train: 0.8410, Val: 0.8704
Epoch: 011, Loss: 0.7074, Train: 0.7044, Val: 0.7402
Epoch: 012, Loss: 0.4962, Train: 0.5045, Val: 0.5488
Epoch: 013, Loss: 0.2545, Train: 0.3340, Val: 0.3694
Epoch: 014, Loss: 0.1116, Train: 0.4476, Val: 0.4251
Epoch: 015, Loss: 0.2003, Train: 0.6275, Val: 0.5780
Epoch: 016, Loss: 0.3938, Train: 0.6074, Val: 0.5598
Epoch: 017, Loss: 0.3689, Train: 0.4438, Val: 0.4206
Epoch: 018, Loss: 0.1969, Train: 0.3292, Val: 0.3498
Epoch: 019, Loss: 0.1084, Train: 0.3788, Val: 

In [75]:
import json
datos_completos = {
    'nodos': nodos.to_dict(orient='records'),
    'aristas': aristas.to_dict(orient='records'),
    'parliamentarian_to_index': parliamentarian_to_index,
    'index_to_parliamentarian': index_to_parliamentarian,
    'embeddings': embeddings.tolist()
}

# Guardar el diccionario completo en un archivo JSON
with open('datos_completos.json', 'w', encoding='utf-8') as f:
    json.dump(datos_completos, f, ensure_ascii=False, indent=4)


In [38]:
# Cargar el archivo JSON con todos los datos
def cargar_datos():
    with open('datos_completos.json', 'r', encoding='utf-8') as f:
        datos_completos = json.load(f)
    
    # Convertir los datos en sus respectivos formatos
    nodos = pd.DataFrame(datos_completos['nodos'])
    aristas = pd.DataFrame(datos_completos['aristas'])
    parliamentarian_to_index = datos_completos['parliamentarian_to_index']
    index_to_parliamentarian = datos_completos['index_to_parliamentarian']
    embeddings = torch.tensor(datos_completos['embeddings'])
    
    return nodos, aristas, parliamentarian_to_index, index_to_parliamentarian, embeddings

# Cargar los datos
nodos, aristas, parliamentarian_to_index, index_to_parliamentarian, embeddings = cargar_datos()


In [76]:
import tkinter as tk
from tkinter import ttk
from sklearn.metrics.pairwise import cosine_similarity

# Obtener los embeddings de los nodos después del entrenamiento
@torch.no_grad()
def obtener_embeddings(data):
    data = data.to(device)
    model.eval()
    embeddings = model.encoder(data.x, data.edge_index)
    return embeddings.cpu().numpy()  # Convertir a numpy array para facilitar cálculos de similitud

# Obtener los embeddings de los nodos de test
embeddings = obtener_embeddings(test_data)
similitud = cosine_similarity(embeddings)

# Crear un mapeo inverso de índices a nombres de parlamentarios
index_to_parliamentarian = {idx: name for name, idx in parliamentarian_to_index.items()}

# Crear un mapeo de nombres a índices de parlamentarios (para facilitar la búsqueda)
parliamentarian_to_index = {name: idx for idx, name in index_to_parliamentarian.items()}

# Función para obtener los nodos más similares por nombre
def nodos_mas_parecidos_con_nombres(nodo_idx, top_k=5):
    similitudes_nodo = similitud[nodo_idx]
    nodos_parecidos = similitudes_nodo.argsort()[::-1][1:top_k+1]  # Obtener los índices de los nodos más parecidos
    nombres_parecidos = [index_to_parliamentarian[nodo] for nodo in nodos_parecidos]  # Convertir índices a nombres
    return nombres_parecidos, similitudes_nodo[nodos_parecidos]

# Función para verificar si hay una arista entre dos nodos y obtener proportion_agreement y sector político del nodo encontrado
def obtener_arista_df(df_aristas, nombre_1, nombre_2):
    relacion = df_aristas[((df_aristas['parliamentarian_1'] == nombre_1) & (df_aristas['parliamentarian_2'] == nombre_2)) |
                          ((df_aristas['parliamentarian_1'] == nombre_2) & (df_aristas['parliamentarian_2'] == nombre_1))]
    if not relacion.empty:
        # Si existe la relación, devolver True, proportion_agreement, y el sector del nodo encontrado (nombre_2)
        proportion_agreement = relacion.iloc[0]['proportion_agreement']
        sector_nodo_encontrado = relacion.iloc[0]['sector_2'] if relacion.iloc[0]['parliamentarian_2'] == nombre_2 else relacion.iloc[0]['sector_1']
        return True, proportion_agreement, sector_nodo_encontrado
    else:
        # Si no existe relación, devolver False y None
        return False, None, None

import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import requests
from io import BytesIO
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import requests
from io import BytesIO

# Función para manejar la consulta desde la interfaz gráfica
# Función para manejar la consulta desde la interfaz gráfica
def consultar_nodo():
    nombre_nodo_consulta = combo.get()
    if nombre_nodo_consulta in parliamentarian_to_index:
        nodo_consulta_idx = parliamentarian_to_index[nombre_nodo_consulta]  # Obtener el índice del nodo consultado
        
        # Obtener el sector político del nodo consultado
        sector_nodo_consulta = aristas.loc[aristas['parliamentarian_1'] == nombre_nodo_consulta, 'sector_1'].values[0]
        resultado_label.config(text=f"Sector político del nodo consultado: {sector_nodo_consulta}")
        
        # Buscar los nodos más similares por nombre
        nodos_parecidos_nombres, similitudes = nodos_mas_parecidos_con_nombres(nodo_consulta_idx, top_k=5)
        
        # Limpiar resultados previos
        resultados_box.delete(1.0, tk.END)

        # Imprimir resultados en la caja de texto
        resultados_box.insert(tk.END, f"Nombres de los nodos más parecidos:\n")
        for i, nodo in enumerate(nodos_parecidos_nombres):
            resultados_box.insert(tk.END, f"{nodo} (Similitud: {similitudes[i]:.4f})\n")
        
        # Mostrar la imagen del parlamentario consultado
        url_imagen = nodos.loc[nodos['parliamentarian_1'] == nombre_nodo_consulta, 'url_imagen_1'].values[0]
        try:
            response = requests.get(url_imagen)
            img_data = response.content
            img = Image.open(BytesIO(img_data))
            img = img.resize((150, 150))  # Redimensionar la imagen
            img_tk = ImageTk.PhotoImage(img)

            # Declarar 'img_label' como global antes de usarla
            global img_label
            if 'img_label' in globals():
                img_label.config(image=img_tk)
                img_label.image = img_tk
            else:
                img_label = tk.Label(root, image=img_tk)
                img_label.image = img_tk
                img_label.pack(pady=10)
        
        except Exception as e:
            print(f"Error cargando la imagen: {e}")
            if 'img_label' in globals():
                img_label.config(image='')
    
    else:
        resultado_label.config(text=f"El nodo con nombre {nombre_nodo_consulta} no existe.")



# Lista de nombres de parlamentarios (simulada, debe venir de tu dataframe original)
nombres_nodos = list(parliamentarian_to_index.keys())

# Crear la ventana principal
root = tk.Tk()
root.title("Consulta de Parlamentarios")
root.geometry("500x700")  # Aumenté el tamaño para incluir la imagen

# Etiqueta principal
label = tk.Label(root, text="Selecciona un parlamentario para consultar", font=("Arial", 14))
label.pack(pady=10)

# Crear un menú desplegable (combobox) para seleccionar parlamentarios
combo = ttk.Combobox(root, values=nombres_nodos, font=("Arial", 12))
combo.pack(pady=10)

# Botón para consultar el nodo seleccionado
boton = tk.Button(root, text="Consultar", font=("Arial", 12), command=consultar_nodo)
boton.pack(pady=10)

# Etiqueta donde se mostrará el nodo consultado
resultado_label = tk.Label(root, text="", font=("Arial", 12))
resultado_label.pack(pady=10)

# Caja de texto donde se mostrarán los resultados
resultados_box = tk.Text(root, height=10, width=60, font=("Arial", 10))
resultados_box.pack(pady=10)

# Ejecutar la ventana
root.mainloop()


Error cargando la imagen: cannot identify image file <_io.BytesIO object at 0x000001EC69448EA0>


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\benja\AppData\Local\Temp\ipykernel_35348\2107952269.py", line 81, in consultar_nodo
    img = Image.open(BytesIO(img_data))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benja\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py", line 3498, in open
    raise UnidentifiedImageError(msg)
PIL.UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000001EC69448EA0>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\benja\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\benja\AppData\Local\Temp\ipykernel_35348\2107952269.py", line 98, in consultar_nodo
    img_label.config(image='')
  File "c:\Users\benja\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1722

In [78]:
import json
import pandas as pd
import torch
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import requests
from io import BytesIO
from sklearn.metrics.pairwise import cosine_similarity

# Cargar los datos desde archivo JSON
def cargar_datos():
    with open('datos_completos.json', 'r', encoding='utf-8') as f:
        datos_completos = json.load(f)
    
    nodos = pd.DataFrame(datos_completos['nodos'])
    aristas = pd.DataFrame(datos_completos['aristas'])
    parliamentarian_to_index = datos_completos['parliamentarian_to_index']
    embeddings = torch.tensor(datos_completos['embeddings'])
    
    return nodos, aristas, parliamentarian_to_index, embeddings

# Función para obtener los nodos más similares por nombre
def nodos_mas_parecidos_con_nombres(similitud, index_to_parliamentarian, nodo_idx, top_k=5):
    similitudes_nodo = similitud[nodo_idx]
    nodos_parecidos = similitudes_nodo.argsort()[::-1][1:top_k+1]
    nombres_parecidos = [index_to_parliamentarian[nodo] for nodo in nodos_parecidos]
    return nombres_parecidos, similitudes_nodo[nodos_parecidos]

# Verificar arista y obtener detalles
def obtener_arista_df(df_aristas, nombre_1, nombre_2):
    relacion = df_aristas[((df_aristas['parliamentarian_1'] == nombre_1) & (df_aristas['parliamentarian_2'] == nombre_2)) |
                          ((df_aristas['parliamentarian_1'] == nombre_2) & (df_aristas['parliamentarian_2'] == nombre_1))]
    if not relacion.empty:
        proportion_agreement = relacion.iloc[0]['proportion_agreement']
        sector_nodo_encontrado = relacion.iloc[0]['sector_2'] if relacion.iloc[0]['parliamentarian_2'] == nombre_2 else relacion.iloc[0]['sector_1']
        return True, proportion_agreement, sector_nodo_encontrado
    else:
        return False, None, None

# Cargar los datos
nodos, aristas, parliamentarian_to_index, embeddings = cargar_datos()

# Calcular similitud de embeddings
similitud = cosine_similarity(embeddings)

# Crear un mapeo inverso de índices a nombres de parlamentarios
index_to_parliamentarian = {idx: name for name, idx in parliamentarian_to_index.items()}

def consultar_nodo():
    nombre_nodo_consulta = combo.get()
    if nombre_nodo_consulta in parliamentarian_to_index:
        nodo_consulta_idx = parliamentarian_to_index[nombre_nodo_consulta]  # Obtener el índice del nodo consultado
        
        # Obtener el sector político del nodo consultado
        sector_nodo_consulta = aristas.loc[aristas['parliamentarian_1'] == nombre_nodo_consulta, 'sector_1'].values[0]
        resultado_label.config(text=f"Sector político del nodo consultado: {sector_nodo_consulta}")
        
        # Buscar los nodos más similares por nombre
        nodos_parecidos_nombres, similitudes = nodos_mas_parecidos_con_nombres(nodo_consulta_idx, top_k=5)
        
        # Limpiar resultados previos
        resultados_box.delete(1.0, tk.END)

        # Imprimir resultados en la caja de texto
        resultados_box.insert(tk.END, f"Nombres de los nodos más parecidos:\n")
        for i, nodo in enumerate(nodos_parecidos_nombres):
            resultados_box.insert(tk.END, f"{nodo} (Similitud: {similitudes[i]:.4f})\n")
        
        # Mostrar la imagen del parlamentario consultado
        url_imagen = nodos.loc[nodos['parliamentarian_1'] == nombre_nodo_consulta, 'url_imagen_1'].values[0]
        try:
            response = requests.get(url_imagen)
            img_data = response.content
            img = Image.open(BytesIO(img_data))
            img = img.resize((150, 150))  # Redimensionar la imagen
            img_tk = ImageTk.PhotoImage(img)

            # Declarar 'img_label' como global antes de usarla
            global img_label
            if 'img_label' in globals():
                img_label.config(image=img_tk)
                img_label.image = img_tk
            else:
                img_label = tk.Label(root, image=img_tk)
                img_label.image = img_tk
                img_label.pack(pady=10)
        
        except Exception as e:
            print(f"Error cargando la imagen: {e}")
            if 'img_label' in globals():
                img_label.config(image='')
    
    else:
        resultado_label.config(text=f"El nodo con nombre {nombre_nodo_consulta} no existe.")

# Lista de nombres de parlamentarios (simulada, debe venir de tu dataframe original)
nombres_nodos = list(parliamentarian_to_index.keys())

# Crear la ventana principal
root = tk.Tk()
root.title("Consulta de Parlamentarios")
root.geometry("500x700")  # Aumenté el tamaño para incluir la imagen

# Etiqueta principal
label = tk.Label(root, text="Selecciona un parlamentario para consultar", font=("Arial", 14))
label.pack(pady=10)

# Crear un menú desplegable (combobox) para seleccionar parlamentarios
combo = ttk.Combobox(root, values=nombres_nodos, font=("Arial", 12))
combo.pack(pady=10)

# Botón para consultar el nodo seleccionado
boton = tk.Button(root, text="Consultar", font=("Arial", 12), command=consultar_nodo)
boton.pack(pady=10)

# Etiqueta donde se mostrará el nodo consultado
resultado_label = tk.Label(root, text="", font=("Arial", 12))
resultado_label.pack(pady=10)

# Caja de texto donde se mostrarán los resultados
resultados_box = tk.Text(root, height=10, width=60, font=("Arial", 10))
resultados_box.pack(pady=10)

# Ejecutar la ventana
root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\benja\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\benja\AppData\Local\Temp\ipykernel_35348\3350612482.py", line 60, in consultar_nodo
    nodos_parecidos_nombres, similitudes = nodos_mas_parecidos_con_nombres(nodo_consulta_idx, top_k=5)
                                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: nodos_mas_parecidos_con_nombres() missing 2 required positional arguments: 'index_to_parliamentarian' and 'nodo_idx'


In [68]:
import json
import pandas as pd
import torch
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import requests
from io import BytesIO
from sklearn.metrics.pairwise import cosine_similarity

# Cargar los datos desde archivo JSON
def cargar_datos():
    with open('datos_completos.json', 'r', encoding='utf-8') as f:
        datos_completos = json.load(f)
    
    nodos = pd.DataFrame(datos_completos['nodos'])
    aristas = pd.DataFrame(datos_completos['aristas'])
    parliamentarian_to_index = datos_completos['parliamentarian_to_index']
    embeddings = torch.tensor(datos_completos['embeddings'])
    
    return nodos, aristas, parliamentarian_to_index, embeddings

# Función para obtener los nodos más similares por nombre
def nodos_mas_parecidos_con_nombres(similitud, index_to_parliamentarian, nodo_idx, top_k=5):
    similitudes_nodo = similitud[nodo_idx]
    nodos_parecidos = similitudes_nodo.argsort()[::-1][1:top_k+1]
    nombres_parecidos = [index_to_parliamentarian[nodo] for nodo in nodos_parecidos]
    return nombres_parecidos, similitudes_nodo[nodos_parecidos]

# Verificar arista y obtener detalles
def obtener_arista_df(df_aristas, nombre_1, nombre_2):
    relacion = df_aristas[((df_aristas['parliamentarian_1'] == nombre_1) & (df_aristas['parliamentarian_2'] == nombre_2)) |
                          ((df_aristas['parliamentarian_1'] == nombre_2) & (df_aristas['parliamentarian_2'] == nombre_1))]
    if not relacion.empty:
        proportion_agreement = relacion.iloc[0]['proportion_agreement']
        sector_nodo_encontrado = relacion.iloc[0]['sector_2'] if relacion.iloc[0]['parliamentarian_2'] == nombre_2 else relacion.iloc[0]['sector_1']
        return True, proportion_agreement, sector_nodo_encontrado
    else:
        return False, None, None

# Cargar los datos
nodos, aristas, parliamentarian_to_index, embeddings = cargar_datos()

# Calcular similitud de embeddings
similitud = cosine_similarity(embeddings)

# Crear un mapeo inverso de índices a nombres de parlamentarios
index_to_parliamentarian = {idx: name for name, idx in parliamentarian_to_index.items()}

# Función para manejar la consulta desde la interfaz gráfica
def consultar_nodo():
    nombre_nodo_consulta = combo.get()
    if nombre_nodo_consulta in parliamentarian_to_index:
        nodo_consulta_idx = parliamentarian_to_index[nombre_nodo_consulta]  # Obtener el índice del nodo consultado
        
        # Obtener el sector político del nodo consultado
        sector_nodo_consulta = aristas.loc[aristas['parliamentarian_1'] == nombre_nodo_consulta, 'sector_1'].values[0]
        resultado_label.config(text=f"Sector político del nodo consultado: {sector_nodo_consulta}")
        
        # Buscar los nodos más similares por nombre
        nodos_parecidos_nombres, similitudes = nodos_mas_parecidos_con_nombres(similitud, index_to_parliamentarian, nodo_consulta_idx, top_k=5)
        
        # Limpiar resultados previos
        resultados_box.delete(1.0, tk.END)

        # Imprimir resultados en la caja de texto
        resultados_box.insert(tk.END, f"Nombres de los nodos más parecidos:\n")
        for i, nodo in enumerate(nodos_parecidos_nombres):
            resultados_box.insert(tk.END, f"{nodo} (Similitud: {similitudes[i]:.4f})\n")
        
        # Mostrar la imagen del parlamentario consultado
        url_imagen = nodos.loc[nodos['parliamentarian_1'] == nombre_nodo_consulta, 'url_imagen_1'].values[0]
        try:
            response = requests.get(url_imagen)
            response.raise_for_status()  # Asegura que no hay errores en la respuesta
            img_data = response.content
            img = Image.open(BytesIO(img_data))
            img = img.resize((150, 150))  # Redimensionar la imagen
            img_tk = ImageTk.PhotoImage(img)

            # Declarar 'img_label' como global antes de usarla
            global img_label

            # Verificar si 'img_label' ya existe y es un Label válido
            if 'img_label' in globals() and isinstance(img_label, tk.Label):
                # Si el widget 'img_label' ya existe, actualizar la imagen
                img_label.config(image=img_tk)
                img_label.image = img_tk
            else:
                # Si no existe, crear el widget 'Label' con la imagen
                img_label = tk.Label(root, image=img_tk)
                img_label.image = img_tk
                img_label.pack(pady=10)
        
        except requests.exceptions.RequestException as e:
            print(f"Error cargando la imagen: {e}")
            # Si ocurre un error, asegurarse de que 'img_label' no tenga una imagen inválida
            if 'img_label' in globals() and isinstance(img_label, tk.Label):
                img_label.config(image='')
    
    else:
        resultado_label.config(text=f"El nodo con nombre {nombre_nodo_consulta} no existe.")

# Lista de nombres de parlamentarios (simulada, debe venir de tu dataframe original)
nombres_nodos = list(parliamentarian_to_index.keys())

# Crear la ventana principal
root = tk.Tk()
root.title("Consulta de Parlamentarios")
root.geometry("500x700")  # Aumenté el tamaño para incluir la imagen

# Etiqueta principal
label = tk.Label(root, text="Selecciona un parlamentario para consultar", font=("Arial", 14))
label.pack(pady=10)

# Crear un menú desplegable (combobox) para seleccionar parlamentarios
combo = ttk.Combobox(root, values=nombres_nodos, font=("Arial", 12))
combo.pack(pady=10)

# Botón para consultar el nodo seleccionado
boton = tk.Button(root, text="Consultar", font=("Arial", 12), command=consultar_nodo)
boton.pack(pady=10)

# Etiqueta donde se mostrará el nodo consultado
resultado_label = tk.Label(root, text="", font=("Arial", 12))
resultado_label.pack(pady=10)

# Caja de texto donde se mostrarán los resultados
resultados_box = tk.Text(root, height=10, width=60, font=("Arial", 10))
resultados_box.pack(pady=10)

# Ejecutar la ventana
root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\benja\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\benja\AppData\Local\Temp\ipykernel_35348\496516582.py", line 87, in consultar_nodo
    img_label.config(image=img_tk)
  File "c:\Users\benja\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1722, in configure
    return self._configure('configure', cnf, kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benja\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1712, in _configure
    self.tk.call(_flatten((self._w, cmd)) + self._options(cnf))
_tkinter.TclError: invalid command name ".!label3"


In [41]:
import json
import pandas as pd
import torch
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import requests
from io import BytesIO
from sklearn.metrics.pairwise import cosine_similarity

# Cargar los datos desde archivo JSON
def cargar_datos():
    with open('datos_completos.json', 'r', encoding='utf-8') as f:
        datos_completos = json.load(f)
    
    nodos = pd.DataFrame(datos_completos['nodos'])
    aristas = pd.DataFrame(datos_completos['aristas'])
    parliamentarian_to_index = datos_completos['parliamentarian_to_index']
    embeddings = torch.tensor(datos_completos['embeddings'])
    
    return nodos, aristas, parliamentarian_to_index, embeddings

# Función para obtener los nodos más similares por nombre
def nodos_mas_parecidos_con_nombres(similitud, index_to_parliamentarian, nodo_idx, top_k=5):
    similitudes_nodo = similitud[nodo_idx]
    nodos_parecidos = similitudes_nodo.argsort()[::-1][1:top_k+1]
    nombres_parecidos = [index_to_parliamentarian[nodo] for nodo in nodos_parecidos]
    return nombres_parecidos, similitudes_nodo[nodos_parecidos]

# Verificar arista y obtener detalles
def obtener_arista_df(df_aristas, nombre_1, nombre_2):
    relacion = df_aristas[((df_aristas['parliamentarian_1'] == nombre_1) & (df_aristas['parliamentarian_2'] == nombre_2)) |
                          ((df_aristas['parliamentarian_1'] == nombre_2) & (df_aristas['parliamentarian_2'] == nombre_1))]
    if not relacion.empty:
        proportion_agreement = relacion.iloc[0]['proportion_agreement']
        sector_nodo_encontrado = relacion.iloc[0]['sector_2'] if relacion.iloc[0]['parliamentarian_2'] == nombre_2 else relacion.iloc[0]['sector_1']
        return True, proportion_agreement, sector_nodo_encontrado
    else:
        return False, None, None

# Cargar los datos
nodos, aristas, parliamentarian_to_index, embeddings = cargar_datos()

# Calcular similitud de embeddings
similitud = cosine_similarity(embeddings)

# Crear un mapeo inverso de índices a nombres de parlamentarios
index_to_parliamentarian = {idx: name for name, idx in parliamentarian_to_index.items()}

def consultar_nodo():
    nombre_nodo_consulta = combo.get()
    if nombre_nodo_consulta in parliamentarian_to_index:
        nodo_consulta_idx = parliamentarian_to_index[nombre_nodo_consulta]  # Obtener el índice del nodo consultado
        
        # Obtener el sector político del nodo consultado
        sector_nodo_consulta = aristas.loc[aristas['parliamentarian_1'] == nombre_nodo_consulta, 'sector_1'].values[0]
        resultado_label.config(text=f"Sector político del nodo consultado: {sector_nodo_consulta}")
        
        # Buscar los nodos más similares por nombre
        nodos_parecidos_nombres, similitudes = nodos_mas_parecidos_con_nombres(similitud, index_to_parliamentarian, nodo_consulta_idx, top_k=5)
        
        # Limpiar resultados previos
        resultados_box.delete(1.0, tk.END)

        # Imprimir resultados en la caja de texto
        resultados_box.insert(tk.END, f"Nombres de los nodos más parecidos:\n")
        for i, nodo in enumerate(nodos_parecidos_nombres):
            resultados_box.insert(tk.END, f"{nodo} (Similitud: {similitudes[i]:.4f})\n")
        
        # Mostrar la imagen del parlamentario consultado
        url_imagen = nodos.loc[nodos['parliamentarian_1'] == nombre_nodo_consulta, 'url_imagen_1'].values[0]
        try:
            response = requests.get(url_imagen)
            img_data = response.content
            img = Image.open(BytesIO(img_data))
            img = img.resize((150, 150))  # Redimensionar la imagen
            img_tk = ImageTk.PhotoImage(img)

            # Declarar 'img_label' como global antes de usarla
            global img_label
            if 'img_label' in globals():
                img_label.config(image=img_tk)
                img_label.image = img_tk
            else:
                img_label = tk.Label(root, image=img_tk)
                img_label.image = img_tk
                img_label.pack(pady=10)
        
        except Exception as e:
            print(f"Error cargando la imagen: {e}")
            if 'img_label' in globals():
                img_label.config(image='')
    
    else:
        resultado_label.config(text=f"El nodo con nombre {nombre_nodo_consulta} no existe.")

# Lista de nombres de parlamentarios (simulada, debe venir de tu dataframe original)
nombres_nodos = list(parliamentarian_to_index.keys())

# Crear la ventana principal
root = tk.Tk()
root.title("Consulta de Parlamentarios")
root.geometry("500x700")  # Aumenté el tamaño para incluir la imagen

# Etiqueta principal
label = tk.Label(root, text="Selecciona un parlamentario para consultar", font=("Arial", 14))
label.pack(pady=10)

# Crear un menú desplegable (combobox) para seleccionar parlamentarios
combo = ttk.Combobox(root, values=nombres_nodos, font=("Arial", 12))
combo.pack(pady=10)

# Botón para consultar el nodo seleccionado
boton = tk.Button(root, text="Consultar", font=("Arial", 12), command=consultar_nodo)
boton.pack(pady=10)

# Etiqueta donde se mostrará el nodo consultado
resultado_label = tk.Label(root, text="", font=("Arial", 12))
resultado_label.pack(pady=10)

# Caja de texto donde se mostrarán los resultados
resultados_box = tk.Text(root, height=10, width=60, font=("Arial", 10))
resultados_box.pack(pady=10)

# Ejecutar la ventana
root.mainloop()

Error cargando la imagen: invalid command name ".!label3"


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\benja\AppData\Local\Temp\ipykernel_35348\3385843083.py", line 82, in consultar_nodo
    img_label.config(image=img_tk)
  File "c:\Users\benja\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1722, in configure
    return self._configure('configure', cnf, kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benja\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1712, in _configure
    self.tk.call(_flatten((self._w, cmd)) + self._options(cnf))
_tkinter.TclError: invalid command name ".!label3"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\benja\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\benja\AppData\Local\Temp\ipykernel_35348\3385843083.py", line 92, in 

In [42]:
import json
import pandas as pd
import torch
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import requests
from io import BytesIO
from sklearn.metrics.pairwise import cosine_similarity

# Cargar los datos desde archivo JSON
def cargar_datos():
    with open('datos_completos.json', 'r', encoding='utf-8') as f:
        datos_completos = json.load(f)
    
    nodos = pd.DataFrame(datos_completos['nodos'])
    aristas = pd.DataFrame(datos_completos['aristas'])
    parliamentarian_to_index = datos_completos['parliamentarian_to_index']
    embeddings = torch.tensor(datos_completos['embeddings'])
    
    return nodos, aristas, parliamentarian_to_index, embeddings

# Función para obtener los nodos más similares por nombre
def nodos_mas_parecidos_con_nombres(similitud, index_to_parliamentarian, nodo_idx, top_k=5):
    similitudes_nodo = similitud[nodo_idx]
    nodos_parecidos = similitudes_nodo.argsort()[::-1][1:top_k+1]
    nombres_parecidos = [index_to_parliamentarian[nodo] for nodo in nodos_parecidos]
    return nombres_parecidos, similitudes_nodo[nodos_parecidos]

# Verificar arista y obtener detalles
def obtener_arista_df(df_aristas, nombre_1, nombre_2):
    relacion = df_aristas[((df_aristas['parliamentarian_1'] == nombre_1) & (df_aristas['parliamentarian_2'] == nombre_2)) |
                          ((df_aristas['parliamentarian_1'] == nombre_2) & (df_aristas['parliamentarian_2'] == nombre_1))]
    if not relacion.empty:
        proportion_agreement = relacion.iloc[0]['proportion_agreement']
        sector_nodo_encontrado = relacion.iloc[0]['sector_2'] if relacion.iloc[0]['parliamentarian_2'] == nombre_2 else relacion.iloc[0]['sector_1']
        return True, proportion_agreement, sector_nodo_encontrado
    else:
        return False, None, None

# Cargar los datos
nodos, aristas, parliamentarian_to_index, embeddings = cargar_datos()

# Calcular similitud de embeddings
similitud = cosine_similarity(embeddings)

# Crear un mapeo inverso de índices a nombres de parlamentarios
index_to_parliamentarian = {idx: name for name, idx in parliamentarian_to_index.items()}

def consultar_nodo():
    nombre_nodo_consulta = combo.get()
    if nombre_nodo_consulta in parliamentarian_to_index:
        nodo_consulta_idx = parliamentarian_to_index[nombre_nodo_consulta]  # Obtener el índice del nodo consultado
        
        # Obtener el sector político del nodo consultado
        sector_nodo_consulta = aristas.loc[aristas['parliamentarian_1'] == nombre_nodo_consulta, 'sector_1'].values[0]
        resultado_label.config(text=f"Sector político del nodo consultado: {sector_nodo_consulta}")
        
        # Buscar los nodos más similares por nombre
        nodos_parecidos_nombres, similitudes = nodos_mas_parecidos_con_nombres(similitud, index_to_parliamentarian, nodo_consulta_idx, top_k=5)
        
        # Limpiar resultados previos
        resultados_box.delete(1.0, tk.END)

        # Imprimir resultados en la caja de texto
        resultados_box.insert(tk.END, f"Nombres de los nodos más parecidos:\n")
        for i, nodo in enumerate(nodos_parecidos_nombres):
            resultados_box.insert(tk.END, f"{nodo} (Similitud: {similitudes[i]:.4f})\n")
        
        # Mostrar la imagen del parlamentario consultado
        url_imagen = nodos.loc[nodos['parliamentarian_1'] == nombre_nodo_consulta, 'url_imagen_1'].values[0]
        try:
            response = requests.get(url_imagen)
            img_data = response.content
            img = Image.open(BytesIO(img_data))
            img = img.resize((150, 150))  # Redimensionar la imagen
            img_tk = ImageTk.PhotoImage(img)

            # Declarar 'img_label' como global antes de usarla
            global img_label
            if 'img_label' in globals() and img_label.winfo_exists():
                img_label.config(image=img_tk)
                img_label.image = img_tk
            else:
                img_label = tk.Label(root, image=img_tk)
                img_label.image = img_tk
                img_label.pack(pady=10)
        
        except Exception as e:
            print(f"Error cargando la imagen: {e}")
            if 'img_label' in globals() and img_label.winfo_exists():
                img_label.config(image='')
    
    else:
        resultado_label.config(text=f"El nodo con nombre {nombre_nodo_consulta} no existe.")

# Lista de nombres de parlamentarios (simulada, debe venir de tu dataframe original)
nombres_nodos = list(parliamentarian_to_index.keys())

# Crear la ventana principal
root = tk.Tk()
root.title("Consulta de Parlamentarios")
root.geometry("500x700")  # Aumenté el tamaño para incluir la imagen

# Etiqueta principal
label = tk.Label(root, text="Selecciona un parlamentario para consultar", font=("Arial", 14))
label.pack(pady=10)

# Crear un menú desplegable (combobox) para seleccionar parlamentarios
combo = ttk.Combobox(root, values=nombres_nodos, font=("Arial", 12))
combo.pack(pady=10)

# Botón para consultar el nodo seleccionado
boton = tk.Button(root, text="Consultar", font=("Arial", 12), command=consultar_nodo)
boton.pack(pady=10)

# Etiqueta donde se mostrará el nodo consultado
resultado_label = tk.Label(root, text="", font=("Arial", 12))
resultado_label.pack(pady=10)

# Caja de texto donde se mostrarán los resultados
resultados_box = tk.Text(root, height=10, width=60, font=("Arial", 10))
resultados_box.pack(pady=10)

# Ejecutar la ventana
root.mainloop()

Error cargando la imagen: can't invoke "winfo" command: application has been destroyed


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\benja\AppData\Local\Temp\ipykernel_35348\2887229492.py", line 81, in consultar_nodo
    if 'img_label' in globals() and img_label.winfo_exists():
                                    ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benja\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1140, in winfo_exists
    self.tk.call('winfo', 'exists', self._w))
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
_tkinter.TclError: can't invoke "winfo" command: application has been destroyed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\benja\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\benja\AppData\Local\Temp\ipykernel_35348\2887229492.py", line 91, in consultar_nodo
    if 'img_label' in globals() and img_label.winf